In [11]:
import pandas as pd
import numpy as np
import math
import pickle

from scipy import stats
import scipy.io
from scipy.spatial.distance import pdist
from scipy.linalg import cholesky
from scipy.io import loadmat

import matlab.engine as engi
import matlab as mat

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,roc_auc_score,recall_score,precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from pyearth import Earth

from src import SMOTE
from src import CFS
from src import metrices_V2 as metrices

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path

import matplotlib.pyplot as plt

In [12]:
def load_data(project):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    commit_guru_path = 'data/commit_guru/' + project + '.csv'
    understand_df = pd.read_csv(understand_path)
    understand_df = understand_df.dropna(axis = 1,how='all')
    cols_list = understand_df.columns.values.tolist()
    for item in ['Kind', 'Name','commit_hash', 'Bugs']:
        if item in cols_list:
            cols_list.remove(item)
            cols_list.insert(0,item)
    understand_df = understand_df[cols_list]
    commit_guru_df = pd.read_csv(commit_guru_path)
    cols = understand_df.columns.tolist()
    
    commit_guru_df = commit_guru_df.drop(labels = ['parent_hashes','author_name','author_name',
                                                   'author_email','fileschanged','author_date',
                                                   'author_date_unix_timestamp', 'commit_message',
                                                  'classification', 'fix', 'contains_bug','fixes',],axis=1)

    understand_df = understand_df.drop_duplicates(cols[4:len(cols)])
    df = understand_df.merge(commit_guru_df,on='commit_hash')
    cols = df.columns.tolist()
    cols = cols[1:] + [cols[0]]
    df = df[cols]
    for item in ['Kind', 'Name','commit_hash']:
        if item in cols:
            df = df.drop(labels = [item],axis=1)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.to_csv('data/converted/'+ project + '_understand.csv',index=False)
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    cols = X.columns
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    return X,y

def apply_smote(df):
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def apply_cfs(df):
        y = df.Bugs.values
        X = df.drop(labels = ['Bugs'],axis = 1)
        X = X.values
        selected_cols = CFS.cfs(X,y)
        cols = df.columns[[selected_cols]].tolist()
        cols.append('Bugs')
        return df[cols],cols

In [30]:
def run_self(project):
    X,y = load_data(project)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
    loc = X_test.CountLineCode
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf =  GaussianNB()
    clf.fit(X_train,y_train)
    importance = 0
    predicted = clf.predict(X_test)
    abcd = metrices.measures(y_test,predicted,loc)
    pf = abcd.get_pf()
    recall = abcd.calculate_recall()
    precision = abcd.calculate_precision()
    f1 = abcd.calculate_f1_score()
    g_score = abcd.get_g_score()
    pci_20 = abcd.get_pci_20()
    ifa = abcd.get_ifa()
    try:
        auc = roc_auc_score(y_test, predicted)
    except:
        auc = 0
    print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance

In [8]:
def run_self_CFS(project):
    X,y = load_data(project)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
    loc = X_test.CountLineCode
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    df_smote,cols = apply_cfs(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf = RandomForestClassifier()
    clf.fit(X_train,y_train)
    importance = clf.feature_importances_
    predicted = clf.predict(X_test[cols[:-1]])
    abcd = metrices.measures(y_test,predicted,loc)
    pf = abcd.get_pf()
    recall = abcd.calculate_recall()
    precision = abcd.calculate_precision()
    f1 = abcd.calculate_f1_score()
    g_score = abcd.get_g_score()
    pci_20 = abcd.get_pci_20()
    ifa = abcd.get_ifa()
    try:
        auc = roc_auc_score(y_test, predicted)
    except:
        auc = 0
    print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance

In [9]:
proj_df = pd.read_csv('projects.csv')
projects = proj_df.repo_name.tolist()

In [31]:
precision_list = {}
recall_list = {}
pf_list = {}
f1_list = {}
g_list = {}
auc_list = {}
pci_20_list = {}
ifa_list = {}
featue_importance = {}
for project in projects:
    try:
        if project == '.DS_Store':
            continue
#         if project != 'org.alloytools.alloy':
#             continue
#         if project != 'guice':
#             continue
        print("+++++++++++++++++   "  + project + "  +++++++++++++++++")
        recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance = run_self(project)
        recall_list[project] = recall
        precision_list[project] = precision
        pf_list[project] = pf
        f1_list[project] = f1
        g_list[project] = g_score
        auc_list[project] = auc
        pci_20_list[project] = pci_20
        ifa_list[project] = ifa
        featue_importance[project] = importance
    except Exception as e:
        print(e)
        continue
final_result = {}
final_result['precision'] = precision_list
final_result['recall'] = recall_list
final_result['pf'] = pf_list
final_result['f1'] = f1_list
final_result['g'] = g_list
final_result['auc'] = auc_list
final_result['pci_20'] = pci_20_list
final_result['ifa'] = ifa_list
with open('results/Performance/process+product_nb.pkl', 'wb') as handle:
    pickle.dump(final_result, handle, protocol=pickle.HIGHEST_PROTOCOL)

+++++++++++++++++   org.alloytools.alloy  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.65      0.97      0.78       138
           1       0.96      0.57      0.71       168

    accuracy                           0.75       306
   macro avg       0.80      0.77      0.74       306
weighted avg       0.82      0.75      0.74       306

+++++++++++++++++   qpython  +++++++++++++++++
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       164

    accuracy                           1.00       164
   macro avg       1.00      1.00      1.00       164
weighted avg       1.00      1.00      1.00       164

+++++++++++++++++   friendlychat-android  +++++++++++++++++
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00         7

    accuracy                           1.00        10
   macro avg   

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        37

    accuracy                           1.00        37
   macro avg       1.00      1.00      1.00        37
weighted avg       1.00      1.00      1.00        37

+++++++++++++++++   openshift-quickstarts  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.14      0.50      0.22         2
           1       0.80      0.40      0.53        10

    accuracy                           0.42        12
   macro avg       0.47      0.45      0.38        12
weighted avg       0.69      0.42      0.48        12

+++++++++++++++++   http-request  +++++++++++++++++
index 1 is out of bounds for axis 0 with size 1
+++++++++++++++++   NumberProgressBar  +++++++++++++++++
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4

    accuracy                           1.00         4
   macro avg      

              precision    recall  f1-score   support

           0       0.62      0.33      0.43        39
           1       0.82      0.94      0.88       129

    accuracy                           0.80       168
   macro avg       0.72      0.64      0.66       168
weighted avg       0.78      0.80      0.77       168

+++++++++++++++++   HackerRank  +++++++++++++++++
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        64

    accuracy                           1.00        64
   macro avg       1.00      1.00      1.00        64
weighted avg       1.00      1.00      1.00        64

+++++++++++++++++   MifareClassicTool  +++++++++++++++++
              precision    recall  f1-score   support

           0       1.00      0.44      0.62        18
           1       0.17      1.00      0.29         2

    accuracy                           0.50        20
   macro avg       0.58      0.72      0.45        20
weighted avg       0.

              precision    recall  f1-score   support

           0       0.33      0.25      0.29         4
           1       0.57      0.67      0.62         6

    accuracy                           0.50        10
   macro avg       0.45      0.46      0.45        10
weighted avg       0.48      0.50      0.48        10

+++++++++++++++++   Shuffle  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.76      0.87      0.81        15
           1       0.60      0.43      0.50         7

    accuracy                           0.73        22
   macro avg       0.68      0.65      0.66        22
weighted avg       0.71      0.73      0.71        22

+++++++++++++++++   AutoEx  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.67      1.00      0.80         4
           1       0.00      0.00      0.00         2

    accuracy                           0.67         6
   macro avg       0.33      0.50  

              precision    recall  f1-score   support

           0       1.00      0.60      0.75         5
           1       0.50      1.00      0.67         2

    accuracy                           0.71         7
   macro avg       0.75      0.80      0.71         7
weighted avg       0.86      0.71      0.73         7

+++++++++++++++++   desertplaceholder  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.56      1.00      0.71         5

    accuracy                           0.56         9
   macro avg       0.28      0.50      0.36         9
weighted avg       0.31      0.56      0.40         9

+++++++++++++++++   Diooto  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.92      0.77      0.84        43
           1       0.38      0.67      0.48         9

    accuracy                           0.75        52
   macro avg       0.65  

...

              precision    recall  f1-score   support

           0       0.71      0.91      0.80      1053
           1       0.58      0.26      0.35       520

    accuracy                           0.69      1573
   macro avg       0.64      0.58      0.58      1573
weighted avg       0.67      0.69      0.65      1573

+++++++++++++++++   javacpp  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.67      0.80      0.73       284
           1       0.62      0.44      0.52       203

    accuracy                           0.65       487
   macro avg       0.64      0.62      0.62       487
weighted avg       0.65      0.65      0.64       487

+++++++++++++++++   XChart  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.65      0.74      0.69       599
           1       0.59      0.47      0.52       464

    accuracy                           0.63      1063
   macro avg       0.62      0.61  

              precision    recall  f1-score   support

           0       0.97      0.20      0.34      2824
           1       0.17      0.97      0.28       464

    accuracy                           0.31      3288
   macro avg       0.57      0.58      0.31      3288
weighted avg       0.86      0.31      0.33      3288

+++++++++++++++++   Jest  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.68      0.82      0.74      1008
           1       0.42      0.26      0.32       516

    accuracy                           0.63      1524
   macro avg       0.55      0.54      0.53      1524
weighted avg       0.60      0.63      0.60      1524

+++++++++++++++++   flume  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.84      0.88      0.86      2022
           1       0.43      0.35      0.38       515

    accuracy                           0.77      2537
   macro avg       0.64      0.62      

              precision    recall  f1-score   support

           0       0.72      0.84      0.77       746
           1       0.46      0.30      0.36       347

    accuracy                           0.67      1093
   macro avg       0.59      0.57      0.57      1093
weighted avg       0.64      0.67      0.64      1093

+++++++++++++++++   streamex  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.70      0.84      0.77       531
           1       0.60      0.40      0.48       312

    accuracy                           0.68       843
   macro avg       0.65      0.62      0.62       843
weighted avg       0.66      0.68      0.66       843

+++++++++++++++++   testcontainers-java  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.74      0.90      0.81       653
           1       0.54      0.28      0.37       286

    accuracy                           0.71       939
   macro avg       0.

              precision    recall  f1-score   support

           0       0.62      0.88      0.72       402
           1       0.75      0.41      0.53       371

    accuracy                           0.65       773
   macro avg       0.68      0.64      0.63       773
weighted avg       0.68      0.65      0.63       773

+++++++++++++++++   spring-cloud-aws  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.60      0.68      0.64       630
           1       0.49      0.41      0.45       474

    accuracy                           0.56      1104
   macro avg       0.55      0.54      0.54      1104
weighted avg       0.55      0.56      0.56      1104

+++++++++++++++++   rtmp-rtsp-stream-client-java  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.90      0.24      0.38       580
           1       0.48      0.96      0.64       419

    accuracy                           0.54       999
   m

              precision    recall  f1-score   support

           0       0.64      0.85      0.73      1268
           1       0.72      0.45      0.55      1112

    accuracy                           0.66      2380
   macro avg       0.68      0.65      0.64      2380
weighted avg       0.68      0.66      0.65      2380

+++++++++++++++++   digdag  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.60      0.86      0.71      1280
           1       0.58      0.25      0.35       989

    accuracy                           0.59      2269
   macro avg       0.59      0.56      0.53      2269
weighted avg       0.59      0.59      0.55      2269

+++++++++++++++++   openrouteservice  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.53      0.72      0.61       864
           1       0.78      0.60      0.68      1412

    accuracy                           0.65      2276
   macro avg       0.65   